In [2]:
import unicodedata
import string
import glob, os

# 0. Setting

## 0-1 read data

In [3]:
def findFiles(path): return glob.glob(path)

In [4]:
all_letters = string.ascii_letters + ".,;"
n_letters = len(all_letters)

In [5]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize("NFD", s)
        if unicodedata.category(c) != 'Mn' 
        and c in all_letters
    )

print(unicodeToAscii('slusarski'))

slusarski


In [6]:
category_lines = {}
all_categories = []

In [7]:
def readLine(filename):
    lines = open(filename, encoding = 'utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

In [8]:
# read data
for filename in findFiles("./data/data/names/*.txt"):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLine(filename)
    category_lines[category] = lines
    
n_categories = len(all_categories)

In [9]:
print(category_lines['Italian'][:5])

['Abandonato', 'Abatangelo', 'Abatantuono', 'Abate', 'Abategiovanni']


# 0-2. convert letter to tensor (one-hot)

In [10]:
import torch

In [11]:
# a -> 0, b -> 1
def letterToIndex(letter):
    return all_letters.find(letter)

In [12]:
# a -> one hot vector 000001 , b -> one hot vector 0000010
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

In [13]:
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

In [14]:
print(lineToTensor('Jone').size())

torch.Size([4, 1, 55])


# 1. create model

In [15]:
import torch.nn as nn
import torch.optim as optim

In [16]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size
        
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim = 1)
        
    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [17]:
n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

# 2. Training

In [18]:
import random

In [19]:
out, hidden = rnn(letterToTensor('A'), torch.zeros(1, n_hidden))

In [20]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1) # topk : 가장 높은 index와 value 추출, 2차원 텐서
    category_i = top_i[0]
    return all_categories[category_i], category_i

In [21]:
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    
    category_tensor = torch.tensor([all_categories.index(category)], dtype = torch.long)
    line_tensor = lineToTensor(line)
    
    return category, line, category_tensor, line_tensor

In [23]:
lr = 0.005
criterion = nn.NLLLoss()
optimizer = optim.SGD(rnn.parameters(), lr = lr)

In [26]:
def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()
    rnn.zero_grad()
    
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)
    
    loss = criterion(output, category_tensor)
    loss.backward()
    optimizer.step()
    
    return output, loss.item()

In [27]:
import time
import math

In [33]:
n_iters = 100000
print_interval = 5000

In [34]:
current_loss = 0
all_losses = []

In [42]:
def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds'%(m,s)

In [45]:
start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss
    
    if iter % print_interval == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = 'O' if guess == category else 'X (%s)' % category
        print('%d %d%% (%s) %.4f %s/ %s %s'%(iter, iter/n_iters * 100, timeSince(start), loss, line, guess, correct))

5000 5% (0m 9s) 1.0921 Alduino/ Italian O
10000 10% (0m 18s) 2.9764 Serafim/ Czech X (Portuguese)
15000 15% (0m 27s) 0.5499 Nakamura/ Japanese O
20000 20% (0m 36s) 2.3370 Araullo/ Italian X (Portuguese)
25000 25% (0m 46s) 0.5540 Yi/ Korean O
30000 30% (0m 56s) 0.3158 Egonidis/ Greek O
35000 35% (1m 6s) 0.4903 Yu/ Korean O
40000 40% (1m 15s) 0.7749 Tze/ Chinese O
45000 45% (1m 25s) 0.6370 Demarchis/ Greek O
50000 50% (1m 34s) 3.2188 Gulia/ Korean X (Russian)
55000 55% (1m 44s) 0.2041 Trang/ Vietnamese O
60000 60% (1m 53s) 0.0781 Loskov/ Russian O
65000 65% (2m 3s) 0.1850 Yoo/ Korean O
70000 70% (2m 13s) 1.6847 Soukup/ Russian X (Czech)
75000 75% (2m 22s) 4.4259 Richard/ Scottish X (Dutch)
80000 80% (2m 32s) 1.9173 Allard/ Arabic X (French)
85000 85% (2m 42s) 0.5466 Estevez/ Spanish O
90000 90% (2m 52s) 1.6725 Thomson/ English X (Scottish)
95000 95% (3m 2s) 1.1680 Dcruz/ Spanish X (Portuguese)
100000 100% (3m 12s) 0.9570 Wallace/ Scottish O


# 3. Test

In [49]:
def evaluate(line_tensor):
    hidden = rnn.initHidden()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    return output

In [55]:
def predict(input_line, n_predictions=3):
    print('\n> %s' % input_line)
    with torch.no_grad():
        output = evaluate(lineToTensor(input_line))

        # Get top N categories
        topv, topi = output.topk(n_predictions, 1, True)
        predictions = []

        for i in range(n_predictions):
            value = topv[0][i].item()
            category_index = topi[0][i].item()
            print('(%.2f) %s' % (value, all_categories[category_index]))
            predictions.append([value, all_categories[category_index]])

In [56]:
predict("nakamura")


> nakamura
(-0.75) Japanese
(-1.93) Arabic
(-2.11) Spanish
